# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Matcher (Finding Classes and Methods
  * Counting Strings used in Classes and Methods and try to find exact matching counter.
  * Find Classes by identifying Strings used only in this single Class.
2. Structure Matcher (Finding Classes)
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.
3. Method Matcher (Find Methods from matching Classes)
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [42]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
%autoreload
from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [44]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# APK Files to load
file_paths = (
    "../../../Downloads/com.snapchat.android_10.86.5.61-2069_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)
file_paths = (
#     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
#     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
    "../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk
Loading Session from Apk at ../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `c_decs` in form of a tuple of Strings
* `m_decs` in form of a tuple of MethodDeclarations
* `f_decs` in form of a tuple of FieldDeclarations

In [45]:
# Classes of the Methods and Fields are automatically merged with c_decs
c_decs = (
    "com.snap.mushroom.MainActivity",
)

c_decs = (
    "com.instagram.mainactivity.MainActivity",
)

In [46]:
m_decs = (
    MethodDec('QE5', 'a', 'QE5', 'PE5'),
    MethodDec('V0h', 'j0', 'IMd'),
    MethodDec('vre', 'g', 'pX4'),
    MethodDec('FOb', '<init>', 'java.lang.String', 'boolean', 'java.lang.String', 'java.lang.String', 'java.lang.String', 'java.lang.Long', 'Xt5', 'long', 'boolean', 'java.lang.Long', 'long')
)
abcd = {
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'): "ACTIVITY_ON_DESTROY",
    MethodDec('android.graphics.Bitmap', 'compress', 'android.graphics.Bitmap$CompressFormat', 'int', 'java.io.OutputStream'): "BITMAP_COMPRESS",
    MethodDec('X.3Pr', 'onLongPress', 'android.view.MotionEvent'): "COMMENT_ONLONGCLICK",
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1D7', 'androidx.fragment.app.FragmentActivity', 'X.0F2', 'android.os.Bundle'): "DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS",
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoryToolbarSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0F2'): "DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORYTOOLBARSWITCHERTOOL",
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0F2'): "DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL",
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'): "DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE",
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0E', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'): "DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME",
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', skip_params=True): "DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE",
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A05', 'com.instagram.direct.visual.DirectVisualMessageViewerController'): "DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW",
    MethodDec('X.3KI', 'onLongPress', 'android.view.MotionEvent'): "DMGESTUREDETECTOR_ONLONGPRESS",
    MethodDec('X.56q', 'A01'): "DMHOLDER_GETCURRENTDMMEDIA",
    MethodDec('X.41v', 'A0E', 'X.41v', 'int'): "DMREPLYCALLBACK_RUN",
    MethodDec('X.41v', 'A0B', 'X.41v'): "DMREPLY_SETSEEN",
    MethodDec('X.1ZQ', 'A00', 'X.0iA'): "FEEDITEM_JSON_GENERATE",
    MethodDec('X.1Ju', 'getActivity'): "FRAGMENT_GETACTIVITY",
    MethodDec('X.0nY', 'A03', 'java.lang.String', 'X.0F2'): "HTTPREQUESTTASKFACTORY_CREATEREQUEST",
    MethodDec('X.65A', 'run'): "INAPPNOTIFICATION_RUN",
    MethodDec('X.7Es', 'A00', 'X.7Es'): "IGTVVIEWERFRAGMENT_GETMEDIA",
    MethodDec('X.7Es', 'A0B', 'X.7Es'): "IGTVVIEWERFRAGMENT_UPDATESTATE",
    MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'): "INDICATEACTIVITY_DO",
    MethodDec('X.0zx', 'close'): "JACKSON_CLOSE",
    MethodDec('X.0hs', 'A04', 'java.io.OutputStream', 'java.lang.Integer'): "JACKSON_CREATEGENERATOR",
    MethodDec('X.0hs', '<init>', 'X.Cnl'): "JACKSON_JSONFACTORY",
    MethodDec('X.1yx', 'A00', 'X.0iz', 'X.1yy'): "LIVE_POST_JSON_GENERATE",
    MethodDec('com.instagram.mainactivity.MainActivity', 'AKg'): "MAINACTIVITY_GETCHILDFRAGMENTMANAGER",
    MethodDec('X.1KR', 'configureActionBar', 'X.1GV'): "MAINFEED_CONFIGUREACTIONBAR",
    MethodDec('X.1RK', 'A00', 'X.0iA', 'boolean'): "MEDIAITEM_JSON_GENERATE",
    MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A01', 'X.0iz', 'X.1RK'): "MEDIA_JSON_GENERATE",
    MethodDec('X.1vx', 'A01'): "NOTIFICATIONINST_GETJSON",
    MethodDec('X.2fy', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'): "NOTIFICATION_SHOW",
    MethodDec('X.39u', 'A01'): "PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN",
    MethodDec('X.1Xg', 'A05', skip_params=True): "POSTDETAIL_SETDATA",
    MethodDec('X.1ma', 'onDoubleTap', 'android.view.MotionEvent'): "POSTGESTURE_SIMPLEPOST_ONDOUBLETAP",
    MethodDec('X.20w', 'onDoubleTap', 'android.view.MotionEvent'): "POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP",
    MethodDec('X.2Ef', 'onDoubleTap', 'android.view.MotionEvent'): "POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP",
    MethodDec('X.1Xm', 'A01', skip_params=True): "POSTVIEW_ADDMEDIA",
    MethodDec('X.2vc', 'A02', skip_params=True): "PROFILEDETAIL_SETDATA",
    MethodDec('X.0kG', 'A05', 'X.0F2', 'X.0ik'): "PROFILE_ISSELF",
    MethodDec('X.2vk', 'A00', skip_params=True): "PROFILE_ONLONGCLICK",
    MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'): "PROXYGEN_SENDREQUESTWITHBODY",
    MethodDec('X.29E', 'A0T', 'int'): "REELVIDEOPLAYER_SEEKTO",
    MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1I', 'X.1nf'): "REELVIEWERFRAGMENT_ADVANCETONEXTREEL",
    MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0o', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'): "REELVIEWERFRAGMENT_PAUSEREEL",
    MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0S', 'com.instagram.reels.fragment.ReelViewerFragment'): "REELVIEWERFRAGMENT_RESUMEREEL",
    MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0z', 'X.1gM', 'X.1nf'): "REELVIEWERFRAGMENT_UPDATEMEDIA",
    MethodDec('X.2ei', 'run'): "REGULARNOTIFICATION_RUN",
    MethodDec('X.0hj', 'A02', 'X.0oT'): "SERIALSCHEDULER_EXECUTE",
    MethodDec('X.0F2', 'getToken'): "SESSION_GETTOKEN",
    MethodDec('X.4J0', 'onFail', 'X.228'): "SIMPLEEXECUTOR_ONFAIL",
    MethodDec('X.4J0', 'onSuccess', 'java.lang.Object'): "SIMPLEEXECUTOR_ONSUCCESS",
    MethodDec('X.4J0', '<init>', 'X.0F2'): "SIMPLEXECUTOR_CONSTRUCTOR",
    MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'): "SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT",
    MethodDec('X.1SK', 'A03', 'X.0iz', 'X.0ij'): "USER_JSON_GENERATE",
    MethodDec('X.1aL', 'A00', 'X.0F2'): "VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED",
    MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int'): "VOLUMEINDICATOR_SETPROGRESS"
}

m_decs = tuple(abcd.keys())

In [47]:
f_decs = tuple()


abcde = {
    FieldDec('X.1GU', 'A08'): 'ACTIONBAR_BASEVIEW',
    FieldDec('X.3Pr', 'A04'): 'COMMENTTOUCHHANDLER_COMMENT',
    FieldDec('X.1UD', 'A0X'): 'COMMENT_TEXT',
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'): 'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER',
    FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'): 'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW',
    FieldDec('X.3KI', 'A01'): 'DMGESTUREDETECTOR_MESSAGEOBJECT',
    FieldDec('X.96J', 'A07'): 'DMHOLDER_MEDIA',
    FieldDec('X.57c', 'A04'): 'DMVOICEMESSAGE_RAWMEDIA',
    FieldDec('X.1ZQ', 'A0I'): 'FEEDITEM_TYPE',
    FieldDec('X.0oS', 'A00'): 'HTTPREQUESTTASK_EXECUTOR',
    FieldDec('X.7Es', 'A0E'): 'IGTVVIEWERFRAGMENT_ROOTVIEW',
    FieldDec('X.1KR', 'A0S'): 'MAINFEED_SESSION',
    FieldDec('X.1Zb', 'A06'): 'MEDIANOTIFICATION_LIST',
    FieldDec('X.1nf', 'A09'): 'MEDIA_WRAPPER_LIVEMEDIAVAR',
    FieldDec('X.1nf', 'A08'): 'MEDIA_WRAPPER_REELMEDIAVAR',
    FieldDec('X.65A', 'A00'): 'NOTIFICATION_INSTANCE',
    FieldDec('X.65A', 'A01'): 'NOTIFICATION_SESSION',
    FieldDec('X.1kQ', 'A05'): 'POSTDETAILUI_PROFILENAME',
    FieldDec('X.1kT', 'A02'): 'POST_VIEW_COMMENTBUTTON',
    FieldDec('X.2vg', 'A0N'): 'PROFILEDETAILUI_BIOTEXT',
    FieldDec('X.2vg', 'A00'): 'PROFILEDETAILUI_FOLLOWER',
    FieldDec('X.2vg', 'A01'): 'PROFILEDETAILUI_FOLLOWING',
    FieldDec('X.2vg', 'A0A'): 'PROFILEDETAILUI_FULLNAME',
    FieldDec('X.2vi', 'A09'): 'PROFILE_PROFILEPIC',
    FieldDec('X.3A0', 'A0D'): 'REELVIDEOPLAYER_CONTROLLER',
    FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1K'): 'REEL_VIEWER_FRAGMENT_SESSION',
    FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'): 'REEL_VIEWER_FRAGMENT_VIDEOPLAYER',
    FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'): 'REEL_VIEWER_FRAGMENT_VIEWROOT',
    FieldDec('X.1nf', 'A0D'): 'STORYUSER_SESSION',
}
f_decs = tuple(abcde.keys())

In [48]:
# Add Classes of Methods and Fields to c_decs
orig_c_decs, c_decs = c_decs, tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + c_decs

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [49]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [50]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
print(str(dec_cas).replace(",", "\n"))
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

{'Lcom/instagram/mainactivity/MainActivity;': <analysis.ClassAnalysis Lcom/instagram/mainactivity/MainActivity;>
 'Landroid/graphics/Bitmap;': <analysis.ClassAnalysis Landroid/graphics/Bitmap; EXTERNAL>
 'LX/3Pr;': <analysis.ClassAnalysis LX/3Pr;>
 'Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;': <analysis.ClassAnalysis Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;>
 'Lcom/instagram/direct/visual/DirectVisualMessageViewerController;': <analysis.ClassAnalysis Lcom/instagram/direct/visual/DirectVisualMessageViewerController;>
 'LX/3KI;': <analysis.ClassAnalysis LX/3KI;>
 'LX/56q;': <analysis.ClassAnalysis LX/56q;>
 'LX/41v;': <analysis.ClassAnalysis LX/41v;>
 'LX/1ZQ;': <analysis.ClassAnalysis LX/1ZQ;>
 'LX/1Ju;': <analysis.ClassAnalysis LX/1Ju;>
 'LX/0nY;': <analysis.ClassAnalysis LX/0nY;>
 'LX/65A;': <analysis.ClassAnalysis LX/65A;>
 'LX/7Es;': <analysis.ClassAnalysis LX/7Es;>
 'Lcom/instagram/realtimeclient/RealtimeClientManager;': <analysis.ClassAnalys

## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [51]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [52]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0z(X.1gM, X.1nf) -> com.instagram.reels.fragment.ReelViewerFragment#A0y(X.1gc, X.1nw)
+ Found single candidate for Method. Considering it a match 
	X.1vx#A01() -> X.1wE#A01()
+ Found single candidate for Method. Considering it a match 
	X.2vc#A02(X.2vg, android.content.Context, X.0F2, X.0ij, X.0Ru, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1RK, java.lang.Integer, boolean, boolean, X.1bM) -> X.2w1#A02(X.2w5, android.content.Context, X.0F2, X.0iu, X.0S6, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1RY, java.lang.Integer, boolean, boolean, X.1ba)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.instagram.realtimeclient.RealtimeClientManager#sendComman

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [53]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/0F2; -> LX/0F2;
+ Found single candidate for Class. Considering it a match! 
	LX/0hj; -> LX/0hu;
+ Found single candidate for Class. Considering it a match! 
	LX/0nY; -> LX/0nj;
+ Found single candidate for Class. Considering it a match! 
	LX/0oS; -> LX/0od;
+ Found single candidate for Class. Considering it a match! 
	LX/0zx; -> LX/108;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer;
+ Found single candidate for Class. Considering it a match! 
	LX/1GU; -> LX/1Gi;
+ Found single candidate for Class. Considering it a match! 
	LX/1Ju; -> LX/1K8;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__Json

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [54]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	Landroid/graphics/Bitmap; -> Landroid/graphics/Bitmap;
+ Found single candidate for Class. Considering it a match! 
	LX/3Pr; -> LX/3QK;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3KI; -> LX/3Kl;
+ Found single candidate for Class. Considering it a match! 
	LX/56q; -> LX/4yp;
+ Found single candidate for Class. Considering it a match! 
	LX/0hs; -> LX/0i3;
+ Found single candidate for Class. Considering it a match! 
	LX/1ma; -> LX/1mr;
* Found multiple candidates for Class LX/20w;
* Found multiple candidates for Class LX/2Ef;
+ Found single candidate for Class. Considering it a match! 
	LX/1Xm; -> LX/1Y0;
+ Found single candidate for Class. Considering it a match! 
	LX/0kG; -> LX/0kR;
+ Found single candidate for Class.

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [55]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.39u#A01()
Class not resolved for Method X.20w#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2Ef#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> com.instagram.mainactivity.MainActivity#onDestroy()
+ Found single candidate for Method. Considering it a match 
	android.graphics.Bitmap#compress(android.graphics.Bitmap$CompressFormat, int, java.io.OutputStream) -> android.graphics.Bitmap#compress(android.graphics.Bitmap$CompressFormat, int, java.io.OutputStream)
+ Found single candidate for Method. Considering it a match 
	X.3Pr#onLongPress(android.view.MotionEvent) -> X.3QK#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.1D7, androidx.fragment.app.F

## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [56]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0ij; -> LX/0iu;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1ZR; -> LX/1Zf;
+ Found single candidate for Class. Considering it a match! 
	LX/1yy; -> LX/1zF;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0oW; -> LX/0oh;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	LX/0QZ; -> LX/0Qi;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/colorfilter/ColorFilterAlp

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [57]:
print(len(accumulator.matching_cs), "/", len(orig_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "\n>", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

61 / 1 Classes were matched
51 / 56 Methods were matched
27 / 29 Fields were matched

Matching Classes:
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• X.1vx -> X.1wE
• X.2vc -> X.2w1
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.41v -> X.42P
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1KR -> X.1Kf
• X.1RK -> X.1RY
• X.1aL -> X.1aZ
• X.29E -> X.29V
• X.2ei -> X.2f7
• X.4J0 -> X.4Io
• X.65A -> X.61P
• X.0F2 -> X.0F2
• X.0hj -> X.0hu
• X.0nY -> X.0nj
• X.0oS -> X.0od
• X.0zx -> X.108
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1GU -> X.1Gi
• X.1Ju -> X.1K8
• com.instagram.feed.media.Media__JsonHelper -> com.instagra

• com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int) 
> com.instagram.ui.widget.volume.VolumeIndicator#A00(int, int)
• X.3KI#onLongPress(android.view.MotionEvent) 
> X.3Kl#onLongPress(android.view.MotionEvent)
• X.1ZQ#A00(X.0iA) 
> X.1Ze#A00(X.0iL)
• X.1Ju#getActivity() 
> X.1K8#getActivity()
• X.1yx#A00(X.0iz, X.1yy) 
> X.1zE#A00(X.0jA, X.1zF)
• com.instagram.feed.media.Media__JsonHelper#A01(X.0iz, X.1RK) 
> com.instagram.feed.media.Media__JsonHelper#A01(X.0jA, X.1RY)
• X.0F2#getToken() 
> X.0F2#getToken()
• X.1SK#A03(X.0iz, X.0ij) 
> X.1SY#A03(X.0jA, X.0iu)

Matching Fields:
• LX/1GU;#A08 -> LX/1Gi;#A08
• LX/3Pr;#A04 -> LX/3QK;#A04
• LX/1UD;#A0X -> LX/1UR;#A0X
• Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#A0D -> Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#A0D
• Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#mRootView -> Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#mRootView
• LX/3KI;#A01 -> 

In [58]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(orig_c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('android.graphics.Bitmap', 'compress', 'android.graphics.Bitmap$CompressFormat', 'int', 'java.io.OutputStream'),
    MethodDec('X.3QK', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1DI', 'androidx.fragment.app.FragmentActivity', 'X.0F2', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoryToolbarSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.Dir

In [59]:
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
f_file = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [60]:
with open(c_file, "r") as f:
    c_txt = f.read()

c_txt = c_txt.replace("ClassDec(", "/* TODO */ ClassDec(")

for c1, c2 in accumulator.matching_cs.items():
    c1, c2 = pretty_format_class(c1), pretty_format_class(c2)
    c_txt = c_txt.replace(f'/* TODO */ ClassDec("{c1}")', f'ClassDec("{c2}")')

In [61]:
import re

with open(m_file, "r") as f:
    m_txt = f.read()

dec_regex = re.compile(r"((MethodDec|ConstructorDec)\()")

m_txt = re.sub(dec_regex, r"/* TODO */ \1", m_txt)

m_dec_regex = re.compile(r"val ([A-Za-z0-9_]+) = (/\* TODO \*/ )?(MethodDec|ConstructorDec)\(\s*([A-Za-z0-9_]+),\s*(\"([A-Za-z0-9_]+)\")?,*\s*(.+)\s*([^)]*)\)", re.MULTILINE)

def replace_dec(name, dec, a1, a2):
    if isinstance(dec, MethodDec):
        for m in m_dec_regex.finditer(m_txt):
            if m.group(1) == name:
                break
        else:
            print("Name not found", name)
            return
        
        
        dec_txt = m.group(0)
        
        # Remove /* TODO */ Comment
        dec_txt = dec_txt.replace(m.group(2), "")
        
        # Method Name
        assert ((m.group(6) is None) and (dec.name == "<init>")) or (m.group(6) == dec.name)
        if m.group(6):
            assert dec_txt.count(m.group(5)) == 1
            dec_txt = dec_txt.replace(m.group(5), f'"{str(a2.name)}"')
        
        # Parameters
        if m.group(8):
            params = m.group(8)
            for p1, p2 in zip(get_pretty_params(str(a1.descriptor)), get_pretty_params(str(a2.descriptor))):
                params = params.replace(f'"{p1}"', f'"{p2}"')
            assert dec_txt.count(m.group(8)) == 1
            dec_txt = dec_txt.replace(m.group(8), params)
        
        return m.group(0), dec_txt
        

for dec in m_decs:
    name = abcd[dec]
    
    for m1, m2 in accumulator.matching_ms.items():
        if dec.equals_ma(m1):
            break
    else:
        print("No match found for MethodDec", dec.pretty_format())
        continue
    
    x = replace_dec(name, dec, m1, m2)
    if x:
        m_txt = m_txt.replace(*x)


Name not found BITMAP_COMPRESS
No match found for MethodDec com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoryToolbarSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2)
No match found for MethodDec com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0F2)
No match found for MethodDec X.39u#A01()
No match found for MethodDec X.20w#onDoubleTap(android.view.MotionEvent)
No match found for MethodDec X.2Ef#onDoubleTap(android.view.MotionEvent)


In [62]:
f_dec_regex = re.compile(r"\/\/ Class (.*)\s*.*val ([A-Za-z0-9_$]+) = (\/\* TODO \*\/ )?VariableDec<(.*)>\(\"([A-Za-z0-9._$]+)\"\)")

with open(f_file, "r") as f:
    f_txt = f.read()
f_txt = f_txt.replace("VariableDec<", "/* TODO */ VariableDec<")

f_txt_lines = f_txt.split("\n")

for f1, f2 in accumulator.matching_fs.items():
    for dec in f_decs:
        if dec.class_name == pretty_format_class(str(f1.get_class_name())) and dec.name == f1.name:
            break
    else:
        raise Exception("Unresolved FieldDec")
    
    val_name = abcde[dec]
    
    for m in f_dec_regex.finditer(f_txt):
        if m.group(2) == val_name:
            break
    else:
        raise Exception(f"Field {val_name} not found")
    
    
    
    if not m.group(3):
        continue
    
    dec_txt = m.group(0)
    
    dec_txt = dec_txt.replace(m.group(3), "")
    
    c_group = m.group(1)
    for c1 in c_group.split(", "):
        c2 = accumulator.matching_cs[FormatClassToJava(c1)]
        c_group = c_group.replace(c1, pretty_format_class(c2))
    dec_txt = dec_txt.replace(m.group(1), c_group)
    
    dec_txt = dec_txt.replace(f'"{m.group(5)}"', f'"{str(f2.name)}"')
    f_txt = f_txt.replace(m.group(0), dec_txt)

In [63]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3QK")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.3Kl")

    override val DM_OBJECT = /* TODO */ ClassDec("X.2xm")

    override val DM_HOLDER = ClassDec("X.4yp")

    override val DM_REPLY = ClassDec("X.42P")

    override val DM_VOICEMESSAGE = ClassDec("X.5Ge")

    override val FEEDITEM = ClassDec("X.1Ze")

    override val FRAGMENT = ClassDec("X.1K8")

    override val HTTPREQUESTTASKFACTORY = ClassDec("X.

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs

object VariableDecsV8: AVariableDecs() {
    // Class X.1Gi
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A08")

    // Class X.3QK
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    // Class X.1UR
    override val COMMENT_TEXT = VariableDec<String>("A0X")

    // Class com.instagram.direct.visual.DirectVisualMessageViewerController
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A0D")

    // Class com.instagram.direct.visual.DirectVisualMessageViewerController
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW = VariableDec<ViewGroup>("mRootView")

    // Class X.3Kl
    override val DMG

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.

Note: This code does obviously not know if you left out the parameters on purpose (a lot of parameter types, or multiple constructors...). Please modify the results for the Methods concerned by using the optional `skip_params=True` argument for `MethodDec`.

```kotlin
fun main() {
    fun MemberDec.formatToGraphGuard() = buildString {
        append("MethodDec('")
        append(classDec.className)
        append("', '")
        append(if (this@formatToGraphGuard is MethodDec) methodName else "<init>")
        append("'")
        for (param in params) {
            append(", '")
            if (param is String)
                append(param)
            else if (param is ClassDec)
                append(param.className)
            else if (param is Class<*>)
                append(param.name)
            else error("Illegal Type for param")
            append("'")
        }
        append(")")
    }
    
    println("m_decs = (")
    val strs = MemberDeclarations::class.java.declaredFields.mapNotNull {
        it.isAccessible = true
        val x = it.get(MemberDeclarations)
        if (x !is MemberDec) return@mapNotNull null

        x.formatToGraphGuard()
    }.joinToString(separator = ",\n\t", prefix = "\t")
    println(strs)
    println(")")
}
```